In [1]:
# import pyfits as pf
# import pylab as plt
# from scipy import optimize
# from scipy.signal import medfilt, find_peaks_cwt
# from scipy.ndimage.filters import minimum_filter, maximum_filter, median_filter, convolve
# from scipy.ndimage.measurements import label
import numpy as np

import myHerpyTools as MHT
reload(MHT)

<module 'myHerpyTools' from 'myHerpyTools.pyc'>

In [2]:
cd /Users/Carlos/Documents/HERMES/reductions/new_start_6.5/HD1581/

/Users/Carlos/Documents/HERMES/reductions/new_start_6.5/HD1581


In [3]:
folder = "herpy_out"

In [4]:
flatFileName = '0_20aug/1/20aug10034.fits'
arcFileName = '0_20aug/1/20aug10052.fits'
objFileName = '0_20aug/1/20aug10053.fits'
extracted_arc1 = MHT.read_NPY(arcFileName, "arc", "px", folder)
extracted_obj1 = MHT.read_NPY(objFileName, "obj", "px", folder)

flatFileName = '1_21aug/1/21aug10047.fits'
arcFileName = '1_21aug/1/21aug10046.fits'
objFileName = '1_21aug/1/21aug10041.fits'
extracted_arc2 = MHT.read_NPY(arcFileName, "arc", "px", folder)
extracted_obj2 = MHT.read_NPY(objFileName, "obj", "px", folder)

flatFileName = '2_22aug/1/22aug10032.fits'
arcFileName = '2_22aug/1/22aug10031.fits'
objFileName = '2_22aug/1/22aug10036.fits'
extracted_arc3 = MHT.read_NPY(arcFileName, "arc", "px", folder)
extracted_obj3 = MHT.read_NPY(objFileName, "obj", "px", folder)

flatFileName = '3_24aug/1/24aug10053.fits'
arcFileName = '3_24aug/1/24aug10054.fits'
objFileName = '3_24aug/1/24aug10058.fits'
extracted_arc4 = MHT.read_NPY(arcFileName, "arc", "px", folder)
extracted_obj4 = MHT.read_NPY(objFileName, "obj", "px", folder)

flatFileName = '4_25aug/1/25aug10039.fits'
arcFileName = '4_25aug/1/25aug10043.fits'
objFileName = '4_25aug/1/25aug10044.fits'
extracted_arc5 = MHT.read_NPY(arcFileName, "arc", "px", folder)
extracted_obj5 = MHT.read_NPY(objFileName, "obj", "px", folder)

herpy_out/arc_20aug10052_px.npy read
herpy_out/obj_20aug10053_px.npy read
herpy_out/arc_21aug10046_px.npy read
herpy_out/obj_21aug10041_px.npy read
herpy_out/arc_22aug10031_px.npy read
herpy_out/obj_22aug10036_px.npy read
herpy_out/arc_24aug10054_px.npy read
herpy_out/obj_24aug10058_px.npy read
herpy_out/arc_25aug10043_px.npy read
herpy_out/obj_25aug10044_px.npy read


In [11]:
import pylab as plt
# for i in range(165, 175):
plt.plot(extracted_obj1[169], label = str(i))
plt.plot(extracted_obj2[169], label = str(i))
plt.plot(extracted_obj3[169], label = str(i))
plt.plot(extracted_obj4[169], label = str(i))
plt.plot(extracted_obj5[170], label = str(i))
# plt.legend()
plt.show()

In [14]:
# import pylab as plt
# plt.plot(extracted_arc1[169])
# plt.plot(extracted_arc2[169])
# plt.plot(extracted_arc3[169])
# plt.plot(extracted_arc4[169])
# plt.plot(extracted_arc5[169])
# plt.plot(extracted_obj1[169])
# plt.plot(extracted_obj2[169])
# plt.plot(extracted_obj3[169])
# plt.plot(extracted_obj4[169])
# plt.plot(extracted_obj5[169])
# plt.show()

In [58]:
#FULL LOOP

#Initialise the 3 output arrays
wlSolutions = []
wlErrors = []
wlPolys = []

#loop over each fibre
for thisFibre in range(extracted_arc.shape[0])[169:170]:
    print 'Fibre',thisFibre

    # get the flux from a single arc
    objectArc = extracted_arc[thisFibre].copy() 
    
    # Template to build model from.
    lineListfFileName = '../linelist_blue_v3.txt'
        
    #Create the model, etc
    thisPoly, thisSolution, thisErr = MHT.make_poly_model_err(objectArc, lineListfFileName)
    
    #append lists
    wlPolys.append(thisPoly)
    wlSolutions.append(thisSolution)
    wlErrors.append(thisErr)

#turn lists into np arrays
wlPolys = np.array(wlPolys)
wlSolutions = np.array(wlSolutions)
wlErrors = np.array(wlErrors)


#         plt.plot(x_dense,gaussian(x_dense,p[0],p[1]))
#         plt.plot(firstSliceX,firstSliceY/np.max(firstSliceY))
#         plt.title(maxIdx)
# #         plt.plot(masterArc[thisLineWl[0]-5:thisLineWl[0]+5])
#         plt.show()


Fibre 169
Shift between template and this fibre arc -1


In [60]:
import glob
import numpy as np

lineList_v3 = np.loadtxt("../lineList_blue_v3.txt")
reload(MHT)
pxValuesEpoch = MHT.px_change_across_epochs(
    lineList_v3, glob.glob1("herpy_out","WLS*"), 
    glob.glob1("herpy_out","arc*"), 169, folder)

Fibre 169 epoch 0
Fibre 169 epoch 1
Fibre 169 epoch 2
Fibre 169 epoch 3
Fibre 169 epoch 4


In [74]:
order = 3
epochPolyFit = []
epochDiffToFit = []
#the offsets of 1 epoch per loop
for i in range(pxValuesEpoch.shape[1]):
    
    polyFilter = [(pxValuesEpoch[:,i]-pxValuesEpoch[:,0])<100]
    
    thisEpochPolyFit = np.polyfit(pxValuesEpoch[:,0][polyFilter],(pxValuesEpoch[:,i]-pxValuesEpoch[:,0])[polyFilter], order)
    thisEpochDiffToFit = np.polynomial.polynomial.polyval(pxValuesEpoch[:,0], thisEpochPolyFit[::-1])-(pxValuesEpoch[:,i]-pxValuesEpoch[:,0])

    epochPolyFit.append(thisEpochPolyFit)
    epochDiffToFit.append(thisEpochDiffToFit)
#     plt.plot(pxValuesEpoch[:,0],pxValuesEpoch[:,i]-pxValuesEpoch[:,0], '.')
#     plt.plot(pxValuesEpoch[:,0],np.polynomial.polynomial.polyval(pxValuesEpoch[:,0], thisEpochPolyFit[::-1]))
#     plt.plot(pxValuesEpoch[:,0],thisEpochDiffToFit, '.')
#     plt.show()

#     print thisEpochDiffToFit





In [28]:
MHT.write_NPY(arcFileName, "pol_f", "", wlPolys, folder =folder)
MHT.write_NPY(arcFileName, "WLS_f", "", wlSolutions, folder =folder)
MHT.write_NPY(arcFileName, "WLERR_f", "", wlErrors, folder =folder)


herpy_out/pol_f_20aug10052_ saved
herpy_out/WLS_f_20aug10052_ saved
herpy_out/WLERR_f_20aug10052_ saved


In [86]:
_,WLS_f = make_WLS_from_polys(wlSolutions[0], epochPolyFit)

In [100]:
write_flux_and_wls(extracted_arc1[170],extracted_arc1[170],WLS_f,0)
write_flux_and_wls(extracted_arc2[170],extracted_arc2[170],WLS_f,1)
write_flux_and_wls(extracted_arc3[170],extracted_arc3[170],WLS_f,2)
write_flux_and_wls(extracted_arc4[170],extracted_arc4[170],WLS_f,3)
write_flux_and_wls(extracted_arc5[170],extracted_arc5[170],WLS_f,4)

In [97]:
plt.plot(WLS_f[0], extracted_arc1[170])
plt.plot(WLS_f[1], extracted_arc2[170])
plt.plot(WLS_f[2], extracted_arc3[170])
plt.plot(WLS_f[3], extracted_arc4[170])
plt.plot(WLS_f[4], extracted_arc5[170])
plt.show()

In [96]:
plt.plot( extracted_arc1[170])
plt.plot(extracted_arc2[170])
plt.plot( extracted_arc3[170])
plt.plot(extracted_arc4[170])
plt.plot( extracted_arc5[170])
plt.show()

In [87]:
WLS_f

[array([ 4713.78407785,  4713.84106035,  4713.8980369 , ...,  4902.17434514,
         4902.2100816 ,  4902.24581461]),
 array([ 4713.78407785,  4713.83928705,  4713.8962644 , ...,  4902.17130479,
         4902.20703963,  4902.24277102]),
 array([ 4713.78407785,  4713.83550778,  4713.89248941, ...,  4902.17449474,
         4902.21023872,  4902.24581461]),
 array([ 4713.79074382,  4713.84773507,  4713.90472036, ...,  4902.18229244,
         4902.21803503,  4902.24581461]),
 array([ 4713.78407785,  4713.83215995,  4713.88912218, ...,  4902.15036513,
         4902.18608537,  4902.22180215])]

In [ ]:
# np.savetxt('HD1581_1.txt',np.vstack((wlSolutions[170], extracted_obj[170])).transpose())
# np.savetxt('ThXe_1.txt',np.vstack((wlSolutions[170], extracted_arc[170])).transpose())

In [ ]:
# np.savetxt('HD1581_0.txt',np.vstack((wlSolutions[170], extracted_obj[170])).transpose())
np.savetxt('ThXe_0.txt',np.vstack((wlSolutions[170], extracted_arc[170])).transpose())

In [ ]:
np.savetxt('HD1581_2.txt',np.vstack((wlSolutions[170], extracted_obj[170])).transpose())
np.savetxt('ThXe_2.txt',np.vstack((wlSolutions[170], extracted_arc[170])).transpose())

In [ ]:
np.savetxt('HD1581_3.txt',np.vstack((wlSolutions[170], extracted_obj[170])).transpose())
np.savetxt('ThXe_3.txt',np.vstack((wlSolutions[170], extracted_arc[170])).transpose())

In [35]:
a = np.load("herpy_out/obj_20aug10053_px.npy")

In [36]:
import pylab as plt
# plt.plot(np.nansum(a, axis=1))
# plt.show()

In [39]:
plt.plot(wlSolutions[0],a[169])
plt.show()

In [ ]:
a[170]

In [32]:
wlSolutions[0]

array([ 4713.84469917,  4713.90134228,  4713.95798064, ...,  4901.98194892,
        4902.01684941,  4902.05174421])

In [40]:
import glob

In [45]:
wlnames = glob.glob1(folder,"WLS_f*")
objnames = glob.glob1(folder,"obj*")

In [52]:
for i in range(5):
    print i 
    a = np.load(folder + "/" + wlnames[i])
    b = np.load(folder + "/" + objnames[i])
    c = b[169]
    print a[0]
#     plt.plot(a[0],c)
#     plt.plot(c)

# plt.show()

0
[ 4713.84469917  4713.90134228  4713.95798064 ...,  4901.98194892
  4902.01684941  4902.05174421]
1
[ 4713.70129958  4713.75887071  4713.81643446 ...,  4902.10906243
  4902.14458502  4902.18010379]
2
[ 4713.82581107  4713.88244273  4713.93906957 ...,  4901.99818081
  4902.03320899  4902.06823184]
3
[ 4713.74130815  4713.79860232  4713.85589009 ...,  4902.14467077
  4902.18018149  4902.21568798]
4
[ 4713.93282506  4713.98897365  4714.04511853 ...,  4902.0616315
  4902.09689277  4902.13214915]
